In [28]:
from os import path
import pandas as pd
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.utils import to_categorical
import yaml
from attrdict import AttrDict
from tqdm import tqdm
from statistics import mean, stdev
from sklearn import svm

Data path

In [4]:
data_path = path.join('..', '..', '..', '..', 'Data', 'UCI HAR Dataset')

Reading hyparameters file

In [5]:
with open('statisticalDataLSTMhparams.yaml') as params_file:
    config = yaml.full_load(params_file)
    hparams = AttrDict(config)

Reading dataset

In [6]:
def get_dataset(data_path, data_type, subset):
    if data_type == 'X':
        return pd.read_csv(path.join(data_path, subset, data_type + '_' + subset + '.txt'), header=None, delim_whitespace=True)
    else:
        return to_categorical(np.array(pd.read_csv(path.join(data_path, subset, data_type + '_' + subset + '.txt'), header=None)[0] - 1))
        # return np.array(pd.read_csv(path.join(data_path, subset, data_type + '_' + subset + '.txt'), header=None)[0])


In [7]:
X_train = get_dataset(data_path, 'X', 'train')
y_train = get_dataset(data_path, 'y', 'train')

In [8]:
X_test = get_dataset(data_path, 'X', 'test')
y_test = get_dataset(data_path, 'y', 'test')

Model Structure

In [ ]:
def build_model(hparams, verbose=True):
    model = keras.models.Sequential()
    model.add(keras.layers.LSTM(hparams.neuron_count, activation="tanh", input_shape=(561, hparams.n_features)))

    # To use Bidirectional LSTM, uncomment the below line and comment the above one.
    # model.add(keras.layers.Bidirectional(keras.layers.LSTM(hparams.neuron_count, activation="tanh"), input_shape=(561, hparams.n_features)))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(hparams.neuron_count, activation="tanh"))
    model.add(keras.layers.Dense(hparams.output_neuron, activation="softmax"))
    model.compile(optimizer="adam", loss="kullback_leibler_divergence", metrics=["accuracy"])
    if(verbose):
        print(model.summary())
    return model

In [30]:
model = build_model(hparams)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 200)               112400    
_________________________________________________________________
dense_9 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_10 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_11 (Dense)             (None, 6)                 306       
Total params: 137,856
Trainable params: 137,856
Non-trainable params: 0
_________________________________________________________________
None


# Train & Test

In [43]:
es = keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=5)
model.fit(X_train, y_train, epochs=40, verbose=True, batch_size=16)

Epoch 1/40
460/460 [==============================] - 2s 4ms/step - loss: 0.0710 - accuracy: 0.9743
Epoch 2/40
460/460 [==============================] - 2s 3ms/step - loss: 0.0626 - accuracy: 0.9773
Epoch 3/40
460/460 [==============================] - 2s 4ms/step - loss: 0.0591 - accuracy: 0.9761
Epoch 4/40
460/460 [==============================] - 2s 5ms/step - loss: 0.0607 - accuracy: 0.9761
Epoch 5/40
460/460 [==============================] - 2s 5ms/step - loss: 0.0623 - accuracy: 0.9762
Epoch 6/40
460/460 [==============================] - 2s 5ms/step - loss: 0.0482 - accuracy: 0.9804
Epoch 7/40
460/460 [==============================] - 2s 5ms/step - loss: 0.0591 - accuracy: 0.9786
Epoch 8/40
460/460 [==============================] - 2s 5ms/step - loss: 0.0389 - accuracy: 0.9849
Epoch 9/40
460/460 [==============================] - 2s 5ms/step - loss: 0.0591 - accuracy: 0.9778
Epoch 10/40
460/460 [==============================] - 2s 5ms/step - loss: 0.0528 - accuracy: 0.9796

In [45]:
model.evaluate(X_test, y_test, verbose=0)

[0.2706083357334137, 0.9484221339225769]

# Stress Test

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=0, patience=5)
accuracies = list()
for i in tqdm(range(10)):
    model = build_model(hparams, verbose=False)
    model.fit(X_train, y_train, epochs=15, verbose=False,  validation_split=0.1, batch_size=64, callbacks=[es])
    result = model.evaluate(X_test, y_test)[1]
    accuracies.append(result)
    print(result)

In [ ]:
print("max: ", max(accuracies) * 100, "\nmin: ", min(accuracies) * 100, "\nmean: ", mean(accuracies), "\nstd: ", stdev(accuracies))